In [8]:
import os
import sys
import time

import dotenv
from IPython.display import HTML
import numpy as np
import pandas as pd
import requests

dotenv.load_dotenv("../.env")

#EVALAP_API_URL = "http://localhost:8000/v1"
EVALAP_API_URL = "https://evalap.etalab.gouv.fr/v1"
EVALAP_API_KEY = os.getenv("EVALAP_API_KEY") 
ALBERT_API_URL = "https://albert.api.etalab.gouv.fr/v1"
ALBERT_API_KEY = os.getenv("ALBERT_API_KEY")
OPENAI_URL = "https://api.openai.com/v1"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
headers = {"Authorization": f"Bearer {EVALAP_API_KEY}"}

In [9]:
import json
import pandas as pd
from pathlib import Path

# Get all JSON files
json_files = Path("_data/LegalBenchRAG/benchmarks").glob("*.json")

# Process files
rows = []
for json_file in json_files:
    data = json.load(open(json_file, 'r', encoding='utf-8'))

    for test in data['tests']:
        rows.append({
            'query': test['query'],
            'output_true': '\n\n'.join([s['answer'] for s in test['snippets']]),
            'snippets': test['snippets'],
            'dataset_name': json_file.stem
        })

# Create DataFrame
df = pd.DataFrame(rows)

# Quick overview
print(f"Total rows: {len(df)}")
print(f"Datasets: {df['dataset_name'].nunique()}")
df

Total rows: 6889
Datasets: 4


,query,output_true,snippets,dataset_name
0,Consider the Marketing Affiliate Agreement bet...,This agreement shall begin upon the date of it...,[{'file_path': 'cuad/CybergyHoldingsInc_201405...,cuad
1,Consider the Marketing Affiliate Agreement bet...,This agreement shall begin upon the date of it...,[{'file_path': 'cuad/CybergyHoldingsInc_201405...,cuad
2,Consider the Marketing Affiliate Agreement bet...,This Agreement may be terminated by either par...,[{'file_path': 'cuad/CybergyHoldingsInc_201405...,cuad
3,Consider the Marketing Affiliate Agreement bet...,This Agreement is accepted by Company in the S...,[{'file_path': 'cuad/CybergyHoldingsInc_201405...,cuad
4,Consider the Marketing Affiliate Agreement bet...,"MA may not assign, sell, lease or otherwise tr...",[{'file_path': 'cuad/CybergyHoldingsInc_201405...,cuad
...,...,...,...,...
6884,Consider VELCO's Non-Disclosure Agreement; Doe...,"For purposes of this Agreement, “BCSI” shall m...",[{'file_path': 'contractnli/VELCO%20NDA%20rev0...,contractnli
6885,Consider VELCO's Non-Disclosure Agreement; Doe...,"The foregoing notwithstanding, the Recipient m...",[{'file_path': 'contractnli/VELCO%20NDA%20rev0...,contractnli
6886,Consider VELCO's Non-Disclosure Agreement; Doe...,5. In the event that the Recipient is required...,[{'file_path': 'contractnli/VELCO%20NDA%20rev0...,contractnli
6887,Consider VELCO's Non-Disclosure Agreement; Doe...,"The foregoing notwithstanding, the Recipient m...",[{'file_path': 'contractnli/VELCO%20NDA%20rev0...,contractnli


In [10]:
# Publish a dataset
dataset = {"name": "LegalBenchRAG", "readme": "see https://github.com/zeroentropy-ai/legalbenchrag"
           , "default_metric" : "judge_precision"
           , "df": df.to_json()}
response = requests.post(f'{EVALAP_API_URL}/dataset', json=dataset, headers=headers)
resp = response.json()
resp

{'name': 'LegalBenchRAG',
 'readme': 'see https://github.com/zeroentropy-ai/legalbenchrag',
 'default_metric': 'judge_precision',
 'columns_map': None,
 'compliance': False,
 'id': 86,
 'created_at': '2025-12-03T22:21:56.709429',
 'size': 6889,
 'columns': ['query', 'output_true', 'snippets', 'dataset_name'],
 'parquet_size': 0,
 'parquet_columns': []}